<h1> Mass Image Rectification </h1>

In [1]:
import glob
import json
import os
from multiprocessing import Pool, pool

import cv2
import matplotlib.pyplot as plt
import numpy as np
import csv

from scipy.io import loadmat
import tables

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


<h1> Load parameters file </h1>

In [2]:
def load_params(params_file):
    params = json.load(open(params_file))
    cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                   params['CameraParameters1']['TangentialDistortion'] + \
                   [params['CameraParameters1']['RadialDistortion'][2]]
    distCoeffs1 = np.array(distCoeffs1)

    distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                   params['CameraParameters2']['TangentialDistortion'] + \
                   [params['CameraParameters2']['RadialDistortion'][2]]
    distCoeffs2 = np.array(distCoeffs2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()

    imageSize = (4096, 3000)
    
    # perform rectification
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, imageSize, R, T, None, None, None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)

    left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)
    
    return left_maps, right_maps

In [3]:
params_file = '/root/alok/data/rectifications/underwater_enclosure_02/stereo_parameters.json'
left_maps, right_maps = load_params(params_file)

<h1> Load all raw stereo image pairs to be rectified </h1>

In [4]:
def remap_and_save(img_path, output_dir, left_maps, right_maps):
    
    new_path = os.path.join(output_dir, os.path.basename(img_path))
    print('Old path: {}'.format(img_path))
    print('New path: {}'.format(new_path))
    
    img = cv2.imread(img_path)
    if 'left' in img_path:
        img_remap = cv2.remap(img, left_maps[0], left_maps[1], cv2.INTER_LANCZOS4)
    elif 'right' in img_path:
        img_remap = cv2.remap(img, right_maps[0], right_maps[1], cv2.INTER_LANCZOS4)
    else:
        raise Exception('Image path does not contain side')
    if not os.path.isdir(os.path.dirname(new_path)):
        os.makedirs(os.path.dirname(new_path))
    if not os.path.isfile(new_path):
        cv2.imwrite(new_path, img_remap)






In [7]:
# specify the directory from which raw stereo frame pairs will be ingested
stereo_image_pairs_base_directory = '/root/alok/data/images/stereo_frame_pairs_test_set/'

# define output directory to which all rectified images will be written
rectified_stereo_image_pairs_base_directory = '/root/alok/data/images/rectified_stereo_frame_pairs_test_set/'

# get list of all stereo frame pairs
left_images = glob.glob('{}/*/input/left_frame.jpg'.format(stereo_image_pairs_base_directory))
right_images = glob.glob('{}/*/input/right_frame.jpg'.format(stereo_image_pairs_base_directory))
stereo_frame_pairs = list(zip(sorted(left_images), sorted(right_images)))

# create list of arguments to be mapped to multiprocessor
arguments_list = []
for sfp in stereo_frame_pairs:
    left_image_path = sfp[0]
    left_output_dir = os.path.dirname(left_image_path).replace(stereo_image_pairs_base_directory, 
                                                               rectified_stereo_image_pairs_base_directory)
    
    left_arguments = [left_image_path, left_output_dir, left_maps, right_maps]
    arguments_list.append(left_arguments)
    
    right_image_path = sfp[1]
    right_output_dir = os.path.dirname(right_image_path).replace(stereo_image_pairs_base_directory, 
                                                                 rectified_stereo_image_pairs_base_directory)
    
    right_arguments = [right_image_path, right_output_dir, left_maps, right_maps]
    arguments_list.append(right_arguments)


<h1> Run rectification en masse </h1>

In [8]:
# run multiple processes to rectify the images en masse (this may take a while depending on the number of images)

p = Pool(processes=20)
results = p.starmap(remap_and_save, arguments_list[:2])

Old path: /root/alok/data/images/stereo_frame_pairs_test_set/0/input/left_frame.jpg
New path: /root/alok/data/images/rectified_stereo_frame_pairs_test_set/0/input/left_frame.jpg
Old path: /root/alok/data/images/stereo_frame_pairs_test_set/0/input/right_frame.jpg
New path: /root/alok/data/images/rectified_stereo_frame_pairs_test_set/0/input/right_frame.jpg
